In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:

from molformer.utils import get_argparse_defaults, read_config
from molformer.model.args import get_parser
from molformer.tokenizer import MolTranBertTokenizer
from molformer.model.attention import TestBert
from molformer.model.args import get_parser as ARGS
# from src.molformer.model.attention import register_attention
from molformer.model.attention_layer import apply_rotary_pos_emb
from molformer.model.attention import get_full_attention
from molformer.utils import calc_attention_tensor, connectivity_matrix, distance_matrix

from attributedict.collections import AttributeDict
import torch 
import pandas as pd
import glob


def get_bert(config, tokenizer):
    bert_model = (
        TestBert(
            config,
            tokenizer.vocab,
            config.seed_path,
            rotate=config.rotate,
            device=config.device,
        )
        .to(config.device)
        .eval()
    )

    tmp_model = torch.load(config.seed_path)["state_dict"]
#     print(tmp_model.keys())
    bert_model.load_state_dict(dict(tmp_model), strict=True)
    return bert_model

### defaults
DEFAULTS = AttributeDict(get_argparse_defaults(ARGS()))

### import|model
# register_attention("linear")
BERT = get_bert(CONFIG, TOKENIZER)

def get_full_model(CHECKPOINT_PATH : str, VOCAB_PATH : str, CONFIG_PATH):
    DEFAULTS = AttributeDict(get_argparse_defaults(ARGS()))
    CONFIG_FILE_PATH = '/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/hparams.yaml'
    CONFIG = read_config(CONFIG_FILE_PATH)
    
    CONFIG.seed_path = CHECKPOINT_PATH
    TOKENIZER = MolTranBertTokenizer(VOCAB_PATH)
    TOKENIZER.name_or_path = VOCAB_PATH
    BERT = get_bert(CONFIG, TOKENIZER)
    return BERT


def agg_attention(smiles : str, nr_of_layers, CONFIG_PATH, VOCAB_PATH):
    for layer in range(nr_of_layers):
        if layer == 0:
            
            attention_tensor = calc_attention_tensor(smiles, layer, CONFIG_PATH, VOCAB_PATH)
        
        else:
            attention_tensor += calc_attention_tensor(smiles, layer, CONFIG_PATH, VOCAB_PATH)
    
    return attention_tensor/nr_of_layers


def r2(x, y):
    x = x.flatten()
    y = y.flatten()
    LR = LinearRegression()
    LR.fit(np.array(x).reshape(-1, 1), y)
    y_pred = LR.predict(np.array(x).reshape(-1, 1))

    return r2_score(y_pred, y)



    



ModuleNotFoundError: No module named 'src'

In [ ]:
DATA_PATH = '/home/mi52qep/proj/molformer_refactor/src/molformer/data/bace/test.csv'
BACE_TEST = pd.read_csv(DATA_PATH).smiles
CHECKPOINT_PATHS = glob.glob("/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/checkpoints/*.ckpt")
VOCAB_PATH = '/home/mi52qep/proj/molformer_refactor/src/molformer/data/bert_vocab.txt'


CONFIG_FILE_PATH = '/home/mi52qep/proj/molformer/data/Pretrained MoLFormer/hparams.yaml'
CONFIG = read_config(CONFIG_FILE_PATH)
CONFIG.seed_path = CHECKPOINT_PATH
CONFIG.canonical = False
CONFIG.mask = False

TEST_SMILES = BACE_TEST[0]

for CHECKPOINT_PATH in CHECKPOINT_PATHS:
    model = get_full_model(CHECKPOINT_PATH, VOCAB_PATH, CONFIG_FILE_PATH)
    attention = agg_attention(TEST_SMILES, 12, CONFIG, VOCAB_PATH)
    distance_matrix = distance_matrix(TEST_SMILES)
    r2 = r2(attention, distance_matrix)
    %%capture
    print(r2)
